In [ ]:
!pip install -q langchain_google_vertexai datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platfor

In [ ]:
!pip install -q ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.3/178.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


In [ ]:
from google.oauth2 import service_account
import google.auth
from langchain_google_vertexai import ChatVertexAI, VertexAIEmbeddings
from ragas.llms import LangchainLLMWrapper
import pandas as pd
import os
from ragas import evaluate

In [ ]:
import gspread

In [ ]:
from ragas.metrics import (
    context_precision,
    answer_relevancy,
    faithfulness,
    context_recall,
    answer_similarity,
    answer_correctness,
)
# from ragas.metrics.critique import harmfulness

# list of metrics we're going to use
metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_similarity,
    answer_correctness,
]

In [ ]:
credentials_path =  "/content/drive/MyDrive/env/tdtuchat-16614553b756.json"
credentials = service_account.Credentials.from_service_account_file(credentials_path)

In [ ]:
vertextai_llm = ChatVertexAI(
    model="gemini-1.5-pro",
    credentials=credentials
)
vertextai_embeddings = VertexAIEmbeddings(
    credentials=credentials, model_name= "textembedding-gecko"
)

In [ ]:


# Assuming amnesty_qa_df is already defined as in your provided code

# Display the full DataFrame with no truncation
# pd.set_option("display.max_rows", None)
# pd.set_option("display.max_columns", None)
# pd.set_option("display.max_colwidth", None)

# View data

In [ ]:
file_path1 = "/content/drive/MyDrive/Data/RAGAS/Questions_System.xlsx"


df = pd.read_excel(file_path1)
df[:5]


,question,contexts,answer,time,version,ground_truth
0,Những ngành học nào đang được đào tạo tại TDTU?,[PHƯƠNG ÁN TUYỂN SINH ĐẠI HỌC NĂM 2022\n\nTrườ...,"Dựa vào ngữ cảnh đã cung cấp, trường Đại học T...",17.94,2,"Trụ sở chính: 19 Nguyễn Hữu Thọ, phường Tân Ph..."
1,Ký túc xá của TDTU có những tiện ích gì và làm...,https://admission.tdtu.edu.vn/thong-tin-ky-tuc...,"{\n ""output"": ""Ký túc xá của trường đại học T...",13.74,2,Ký túc xá TDTU được thiết kế hiện đại và hài h...
2,Những ngành đào tạo chính của trường BKU là gì?,Trường Đại học Bách khoa (HCMUT)\n\nTrường Đại...,"{\n ""output"": ""Trường Đại học Bách khoa (HCMU...",14.92,2,Trường Đại học Bách Khoa - ĐHQG-HCM đào tạo nh...
3,Trường Đại học FPT có bao nhiêu cơ sở trên cả ...,https://university.fpt.edu.vn/truong-dai-hoc-f...,"Trường Đại học FPT có 5 cơ sở trên cả nước, tr...",10.68,2,Trường Đại học FPT hiện có 5 cơ sở trên toàn q...
4,Năm học tại trường Đại học FPT được chia thành...,Năm học tại Đại học FPT được chia thành 3 học ...,Năm học tại Đại học FPT được chia thành 3 học ...,14.92,2,Năm học tại trường Đại học FPT được chia thành...


In [ ]:
df.columns

Index(['question', 'contexts', 'answer', 'time', 'version', 'ground_truth'], dtype='object')

In [ ]:
# prompt: Lấy tất cả các hàng với df['Version'] == 1

# Assuming df is already defined as in your previous code

df1 = df[df['version'] == 1]
df2 = df[df['version'] == 2]



In [ ]:
# prompt: df2_test get column: question, contexts, answer, ground_truth

df2_test = df2[['question', 'contexts', 'answer', 'ground_truth']]
df2_test[:5]


,question,contexts,answer,ground_truth
0,Những ngành học nào đang được đào tạo tại TDTU?,[PHƯƠNG ÁN TUYỂN SINH ĐẠI HỌC NĂM 2022\n\nTrườ...,"Dựa vào ngữ cảnh đã cung cấp, trường Đại học T...","Trụ sở chính: 19 Nguyễn Hữu Thọ, phường Tân Ph..."
1,Ký túc xá của TDTU có những tiện ích gì và làm...,https://admission.tdtu.edu.vn/thong-tin-ky-tuc...,"{\n ""output"": ""Ký túc xá của trường đại học T...",Ký túc xá TDTU được thiết kế hiện đại và hài h...
2,Những ngành đào tạo chính của trường BKU là gì?,Trường Đại học Bách khoa (HCMUT)\n\nTrường Đại...,"{\n ""output"": ""Trường Đại học Bách khoa (HCMU...",Trường Đại học Bách Khoa - ĐHQG-HCM đào tạo nh...
3,Trường Đại học FPT có bao nhiêu cơ sở trên cả ...,https://university.fpt.edu.vn/truong-dai-hoc-f...,"Trường Đại học FPT có 5 cơ sở trên cả nước, tr...",Trường Đại học FPT hiện có 5 cơ sở trên toàn q...
4,Năm học tại trường Đại học FPT được chia thành...,Năm học tại Đại học FPT được chia thành 3 học ...,Năm học tại Đại học FPT được chia thành 3 học ...,Năm học tại trường Đại học FPT được chia thành...


In [ ]:
# prompt: ép cột contexts là kiểu 1 list trong đó là các string

import pandas as pd

# Assuming df2_test is already defined as in your provided code

# Convert the 'contexts' column to a list of strings
df2_test['contexts'] = df2_test['contexts'].apply(lambda x: [str(x)])

# Display the updated DataFrame
df2_test[:5]


<ipython-input-28-d17e2aa53891>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_test['contexts'] = df2_test['contexts'].apply(lambda x: [str(x)])


,question,contexts,answer,ground_truth
0,Những ngành học nào đang được đào tạo tại TDTU?,[[PHƯƠNG ÁN TUYỂN SINH ĐẠI HỌC NĂM 2022\n\nTrư...,"Dựa vào ngữ cảnh đã cung cấp, trường Đại học T...","Trụ sở chính: 19 Nguyễn Hữu Thọ, phường Tân Ph..."
1,Ký túc xá của TDTU có những tiện ích gì và làm...,[https://admission.tdtu.edu.vn/thong-tin-ky-tu...,"{\n ""output"": ""Ký túc xá của trường đại học T...",Ký túc xá TDTU được thiết kế hiện đại và hài h...
2,Những ngành đào tạo chính của trường BKU là gì?,[Trường Đại học Bách khoa (HCMUT)\n\nTrường Đạ...,"{\n ""output"": ""Trường Đại học Bách khoa (HCMU...",Trường Đại học Bách Khoa - ĐHQG-HCM đào tạo nh...
3,Trường Đại học FPT có bao nhiêu cơ sở trên cả ...,[https://university.fpt.edu.vn/truong-dai-hoc-...,"Trường Đại học FPT có 5 cơ sở trên cả nước, tr...",Trường Đại học FPT hiện có 5 cơ sở trên toàn q...
4,Năm học tại trường Đại học FPT được chia thành...,[Năm học tại Đại học FPT được chia thành 3 học...,Năm học tại Đại học FPT được chia thành 3 học ...,Năm học tại trường Đại học FPT được chia thành...


In [ ]:
type(df2_test["contexts"].iloc[0])

str

In [ ]:
from datasets import Dataset, DatasetDict

# Convert your DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df2_test)

In [ ]:
len(result_list)

14

In [ ]:
import time

result_list = []
for i in range(len(dataset)):  # Duyệt từng mẫu để tránh lỗi
    sample = dataset.select(range(i, i + 1))

    try:
        result = evaluate(
            sample,
            metrics=metrics,
            llm=vertextai_llm,
            embeddings=vertextai_embeddings,
        )
        result_list.append(result)
    except Exception as e:
        print(f"Lỗi tại mẫu {i}: {e}")

    time.sleep(5)  # Nghỉ 5 giây giữa các request


Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[37]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[47]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[49]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[53]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[54]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[56]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[59]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[60]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[61]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[0]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[5]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[1]: TimeoutError()


Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[65]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[66]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[71]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[72]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[73]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[77]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[78]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[79]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[83]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[5]: TimeoutError()


Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]

TypeError: unsupported operand type(s) for +: 'int' and 'EvaluationResult'

In [ ]:
result_list

[{'faithfulness': nan, 'answer_relevancy': nan, 'context_recall': 0.2500, 'context_precision': 0.0000, 'semantic_similarity': 0.8207, 'answer_correctness': nan},
 {'faithfulness': 1.0000, 'answer_relevancy': 0.7190, 'context_recall': 0.5000, 'context_precision': 0.0000, 'semantic_similarity': 0.8163, 'answer_correctness': nan},
 {'faithfulness': 0.2222, 'answer_relevancy': 0.6792, 'context_recall': 0.2321, 'context_precision': 0.0000, 'semantic_similarity': 0.8567, 'answer_correctness': 0.2834},
 {'faithfulness': 0.6000, 'answer_relevancy': 0.0000, 'context_recall': 0.8333, 'context_precision': 1.0000, 'semantic_similarity': 0.9236, 'answer_correctness': 0.7309},
 {'faithfulness': 1.0000, 'answer_relevancy': 0.9524, 'context_recall': 0.2000, 'context_precision': 1.0000, 'semantic_similarity': 0.9221, 'answer_correctness': 0.6055},
 {'faithfulness': 1.0000, 'answer_relevancy': 0.6300, 'context_recall': 0.6250, 'context_precision': 0.0000, 'semantic_similarity': 0.9084, 'answer_correctne

In [ ]:
# prompt: file txt save result_list to "/content/drive/MyDrive/Data/RAGAS/

import os

# ... (Your existing code) ...

# Save result_list to a text file
file_path = "/content/drive/MyDrive/Data/RAGAS/result_list.txt"
os.makedirs(os.path.dirname(file_path), exist_ok=True)  # Create directory if it doesn't exist

with open(file_path, "w") as f:
    for item in result_list:
        f.write(str(item) + "\n")  # Write each element of the list to a new line


## Version 1

In [ ]:
df1_test = df1[['question', 'contexts', 'answer', 'ground_truth']]
df1_test = df1_test.dropna()
df1_test

,question,contexts,answer,ground_truth
14,Thủ tướng phạm minh chính,[{'url': 'https://vi.wikipedia.org/wiki/Phạm_M...,Theo thông tin tôi tìm thấy được trên web thì ...,Thủ tướng Phạm Minh Chính sinh ngày 10 tháng 1...
15,Những ngành học nào đang được đào tạo tại TDTU?,Trường Đại học Tôn Đức Thắng (TDTU) dự kiến tu...,Dựa vào truy vấn và ngữ cảnh được cung cấp sau...,"Trụ sở chính: 19 Nguyễn Hữu Thọ, phường Tân Ph..."
16,Ký túc xá của TDTU có những tiện ích gì và làm...,Trường Đại học Tôn Đức Thắng (TDTU)\n\nTrường ...,"Dựa trên các nguồn thông tin cung cấp, dưới đâ...",Ký túc xá TDTU được thiết kế hiện đại và hài h...
17,Những ngành đào tạo chính của trường BKU là gì?,"Trong đề án tuyển sinh năm 2022, ĐH Bách khoa ...","Dựa vào ngữ cảnh đã cung cấp, những ngành đào ...",Trường Đại học Bách Khoa - ĐHQG-HCM đào tạo nh...
18,Trường Đại học FPT có bao nhiêu cơ sở trên cả ...,Giới thiệu về Đại học FPT\n\nTổng quan\n\nĐại ...,"Trường Đại học FPT có 5 cơ sở trên cả nước, ba...",Trường Đại học FPT hiện có 5 cơ sở trên toàn q...
19,Năm học tại trường Đại học FPT được chia thành...,"Học phí 01 mức tiếng Anh chuẩn bị: 11,900,000 ...",Năm học tại trường Đại học FPT được chia thành...,Năm học tại trường Đại học FPT được chia thành...
20,"Năm nay, trường FTU2 có chương trình trao đổi ...","Tổng điểm thi tốt nghiệp THPT năm 2024 từ 24,0...","Dựa vào các nguồn thông tin được cung cấp, dướ...",Trường Đại học Ngoại thương Cơ sở II (FTU2) th...
21,Các ngành đào tạo của Trường Đại học Mở TP.HCM...,III. Lịch sử hình thành và phát triển Trường Đ...,"Dựa vào các nguồn thông tin được cung cấp, dướ...",Trường Đại học Mở TP.HCM cung cấp đa dạng các ...
22,Học phí của OU có cao không?,III. Học phí: (dự kiến) Đại học chương trình đ...,Học phí của OU (Trường Đại học Mở TP. Hồ Chí...,Mức học phí tại Trường Đại học Mở TP.HCM được ...
23,Giới thiệu ông Phạm Minh Chính,"Chiều 5/4, tại Nhà Quốc hội, Quốc hội khóa XIV...",Theo thông tin tôi tìm thấy được trên web thì ...,Ông Phạm Minh Chính sinh ngày 10 tháng 12 năm ...


In [ ]:
df1_test['contexts'] = df1_test['contexts'].apply(lambda x: [str(x)])

# Display the updated DataFrame

In [ ]:
dataset1 = Dataset.from_pandas(df1_test)

In [ ]:
result_list1 = []
for i in range(len(dataset1)):  # Duyệt từng mẫu để tránh lỗi
    sample = dataset1.select(range(i, i + 1))

    try:
        result = evaluate(
            sample,
            metrics=metrics,
            llm=vertextai_llm,
            embeddings=vertextai_embeddings,
        )
        result_list1.append(result)
    except Exception as e:
        print(f"Lỗi tại mẫu {i}: {e}")

    time.sleep(20)  # Nghỉ 5 giây giữa các request


Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
result_list1

[{'faithfulness': 1.0000, 'answer_relevancy': 0.8246, 'context_recall': 0.4286, 'context_precision': 0.0000, 'semantic_similarity': 0.8808, 'answer_correctness': 0.3273},
 {'faithfulness': 0.1667, 'answer_relevancy': 0.0000, 'context_recall': 0.0000, 'context_precision': 0.0000, 'semantic_similarity': 0.7964, 'answer_correctness': 0.1991},
 {'faithfulness': 0.5588, 'answer_relevancy': 0.8880, 'context_recall': 0.0000, 'context_precision': 0.0000, 'semantic_similarity': 0.9022, 'answer_correctness': 0.2255},
 {'faithfulness': 0.9000, 'answer_relevancy': 0.0000, 'context_recall': 0.0714, 'context_precision': 0.0000, 'semantic_similarity': 0.8537, 'answer_correctness': 0.3043},
 {'faithfulness': 1.0000, 'answer_relevancy': 0.9917, 'context_recall': 1.0000, 'context_precision': 1.0000, 'semantic_similarity': 0.9533, 'answer_correctness': 0.9201},
 {'faithfulness': 1.0000, 'answer_relevancy': 0.9425, 'context_recall': 0.2000, 'context_precision': 0.0000, 'semantic_similarity': 0.9165, 'answ

In [ ]:
# prompt: file txt save result_list to "/content/drive/MyDrive/Data/RAGAS/

import os

# ... (Your existing code) ...

# Save result_list to a text file
file_path = "/content/drive/MyDrive/Data/RAGAS/result_list_1_Version1.txt"
os.makedirs(os.path.dirname(file_path), exist_ok=True)  # Create directory if it doesn't exist

with open(file_path, "w") as f:
    for item in result_list1:
        f.write(str(item) + "\n")  # Write each element of the list to a new line
